In [1]:
import pandas as pd
import numpy as np
import csv
from nltk.stem import WordNetLemmatizer
from gensim.models.doc2vec import TaggedDocument

C:\LearnPython\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
wnl = WordNetLemmatizer()

class Sentences(object):
    
    def __init__(self, filename, column):
        self.filename = filename
        self.column = column
        
    @staticmethod
    def get_tokens(text):
        """Helper function for tokenizing data"""
        return [wnl.lemmatize(r.lower()) for r in text.split()]
 
    def __iter__(self):
        reader = csv.DictReader(open(self.filename, 'r' ))
        for row in reader:
            words = self.get_tokens(row[self.column])
            tags = ['%s|%s' % (row['Artist'], row['SongID'])]
            yield TaggedDocument(words=words, tags=tags)

In [6]:
filename = 'C:/Users/alienware/Desktop/660project/preprocess/lyrics_dfclean_rap100.csv'
sentences = Sentences(filename=filename, column='Lyrics')

# for song lookups
df_train = pd.read_csv(filename)

In [7]:

from gensim.models.doc2vec import Doc2Vec

model = Doc2Vec(
    alpha=0.025,
    min_alpha=0.025,
    workers=15, 
    min_count=2,
    window=10,
    size=300,
    iter=20,
    sample=0.001,
    negative=5
)

C:\LearnPython\lib\site-packages\gensim\models\doc2vec.py:362: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
C:\LearnPython\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [8]:
model.build_vocab(sentences)

In [9]:
model.save('rap-lyrics.doc2vec')

model = Doc2Vec.load('rap-lyrics.doc2vec')

In [10]:
model.wv.word_vec('rap')

array([-8.27293610e-04,  1.13599829e-03, -1.09892178e-04,  6.10313553e-04,
       -1.42151793e-03, -1.59448595e-03, -8.58863350e-04, -1.62838679e-03,
        1.02682493e-03,  1.14852143e-03, -1.06908998e-03,  2.92785146e-04,
       -4.38364776e-04,  5.53547579e-04,  3.11632088e-04,  1.26060928e-04,
       -1.08255947e-04, -3.11860320e-04,  1.59630156e-03,  1.28818268e-03,
       -3.66141030e-04, -7.63866294e-04,  4.97325440e-04,  5.62938978e-04,
       -5.95003599e-04, -2.46542157e-04, -1.55941725e-05,  1.40561839e-03,
        6.02122105e-04,  7.96552456e-04, -9.64202394e-04, -1.16286380e-03,
       -1.66243582e-03,  1.37317274e-03, -1.34061812e-03, -1.02661818e-03,
        1.41749752e-03, -1.30592950e-03,  9.32906463e-04,  1.46190519e-03,
        1.66141323e-03, -1.04556893e-04, -1.01580482e-03, -4.48227482e-04,
       -3.05193134e-05, -7.42997508e-04, -1.42817886e-03, -1.48699665e-03,
        1.50756084e-03,  5.68806136e-04,  4.01450932e-04,  1.58729020e-03,
       -1.32646179e-03, -

In [11]:
model.wv.most_similar('you')

[('starstudded', 0.17968499660491943),
 ('zoovie', 0.1700185090303421),
 ('geeked', 0.16624122858047485),
 ('yeahnfeel', 0.16494768857955933),
 ('toynbout', 0.16405844688415527),
 ('ferg', 0.15051430463790894),
 ('pinky', 0.1471334993839264),
 ('storenhad', 0.1465548276901245),
 ('beyonc茅', 0.14644530415534973),
 ('begginnfor', 0.14447489380836487)]

In [12]:
def print_titles(results):
    lookup = lambda x: df_train[
        df_train.song_id==int(x)
    ].song.values[0]
    return [
        [
            i[0].split('|')[0], 
            lookup(i[0].split('|')[1]), 
            i[1]
        ] for i in results
    ]

In [13]:
print_titles(
    model.docvecs.most_similar([model['??????']], topn=10)
)

KeyError: "tag '??????' not seen in training corpus/invalid"